In [1]:
import pandas as pd
import psycopg2 as pg
import sys
import os
import ipywidgets as w
from elasticsearch import Elasticsearch
import elasticsearch.helpers
from pandasticsearch import Select
import pandasticsearch as pds
con = pg.connect(os.getenv('RGDSN'))
es_con = Elasticsearch('http://es01:9200')

In [25]:
%%html
<style>
.padded {
    /*width: 100px;*/
    text-align: right;
    padding: 0 10px 0 10px;
}
.big {
    font-size: 2em;
    font-weight:bold;
    height: 5em;
}
</style>

In [21]:
r_label = w.Label('0')
r_label.add_class('big')
a_label = w.Label('0')
a_label.add_class('big')
ro_label = w.Label('0')
ro_label.add_class('big')
hb = w.HBox([
    w.VBox([r_label,w.Label('rubrics')], layout=w.Layout(border='10px solid transparent')),
    w.VBox([ro_label,w.Label('rubrics_objects')]),
    w.VBox([a_label,w.Label('articles')]),
])
hb
# rubrics_label

In [24]:
def get_counts():
    """ Возвращает число записей в таблицах Postgress и индексах Elastic"""
    counts = {
        "PG: rubrics" : pd.read_sql_query('SELECT count(1) FROM rubrics', con)['count'][0],
        "PG: rubrics_objects" : pd.read_sql_query('SELECT count(1) FROM rubrics_objects', con)['count'][0],
        "PG: articles" : pd.read_sql_query('SELECT count(1) FROM articles', con)['count'][0],
        "articles: migration_status = 'error'" : pd.read_sql_query("SELECT count(1) FROM articles WHERE migration_status = 'error'", con)['count'][0],
        "articles: process_status   = 'error'" : pd.read_sql_query("SELECT count(1) FROM articles WHERE process_status = 'error'", con)['count'][0],
        "articles: process_status IS NULL    " : pd.read_sql_query("SELECT count(1) FROM articles WHERE process_status IS NULL", con)['count'][0],
        "articles: process_status   = 'success'" : pd.read_sql_query("SELECT count(1) FROM articles WHERE process_status = 'success'", con)['count'][0],
        "articles: elastic_status   = 'indexed'" : pd.read_sql_query("SELECT count(1) FROM articles WHERE elastic_status = 'indexed'", con)['count'][0],
        "ES: articles" : es_con.count(index='articles')['count'],
    }
    return counts

counts = get_counts()
pd.DataFrame.from_dict(counts,orient='index', columns=['count'])

,count
PG: rubrics,1151
PG: rubrics_objects,3117129
PG: articles,1226058
articles: migration_status = 'error',3
articles: process_status = 'error',0
articles: process_status IS NULL,3
articles: process_status = 'success',1226055
articles: elastic_status = 'indexed',1226055
ES: articles,1226055


In [ ]:
df = pd.read_sql('SELECT obj_id, lemmatized_text FROM articles limit 20000', con)
# display(df)
mem = df.memory_usage(deep=True).sum()/(1024*1024)
print(f'mem={mem:.2f} Gb')
# df.loc[0:2,['title','id']]

In [5]:
sys.getsizeof(df)/1024/1024

5228.256238937378

In [4]:
try:
    cur = con.cursor()
    cur.execute('select count(*) from rubrics limit 10')
    recs = cur.fetchall()
    print(recs[0][0])
except Exception as ex:
    print(ex)
finally:
    if cur: cur.close()
#     if con: con.close()

1151


In [68]:
a_label.value = '10'

In [2]:
es = Elasticsearch('http://es01:9200')
# es.count('rubrics')
body={"query": {"match_all": {}}}
body = {  "size" : 1000,  "_source" : ["id"]}

# results = elasticsearch.helpers.scan(es, query=body, index="articles")

df = pd.DataFrame.from_dict([document['_source'] for document in results])
df

KeyboardInterrupt: 

In [58]:
%%time
# https://elasticsearch-py.readthedocs.io/en/7.9.1/xpack.html#sql-apis
res = es.sql.query({"query": "select obj_id from articles", "fetch_size":10000 })
print(len(res['rows']))
cursor = res.get('cursor')
print(cursor)

res = es.sql.query({"cursor": cursor})
print(len(res['rows']))
cursor = res.get('cursor')
print(cursor)

res1 = "didn't try"
if (cursor):
    res1 = es.sql.clear_cursor({"cursor":cursor})

print(f'res1={res1}')

10000
k8OvAwFaAXN4RkdsdVkyeDFaR1ZmWTI5dWRHVjRkRjkxZFdsa0RYRjFaWEo1UVc1a1JtVjBZMmdCRkdZeFNFUnJTRlZDVkVoYVZscFlXRGhCTUVwMkFBQUFBQUFCcngwV1pXaEhRWFpTVEhsUmRGTXdMWFZuTkVzdE1YTk1RUT09/////w8BAWYGb2JqX2lkAQZvYmpfaWQBBHRleHQAAAABAQ==
10000
k8OvAwFaAXN4RkdsdVkyeDFaR1ZmWTI5dWRHVjRkRjkxZFdsa0RYRjFaWEo1UVc1a1JtVjBZMmdCRkdZeFNFUnJTRlZDVkVoYVZscFlXRGhCTUVwMkFBQUFBQUFCcngwV1pXaEhRWFpTVEhsUmRGTXdMWFZuTkVzdE1YTk1RUT09/////w8BAWYGb2JqX2lkAQZvYmpfaWQBBHRleHQAAAABAQ==
res1={'succeeded': True}
CPU times: user 21.1 ms, sys: 4.17 ms, total: 25.3 ms
Wall time: 3.19 s


In [28]:
result_dict = es.search(index="articles", body={"size":9000, "_source" : ["obj_id"]})
print(len(result_dict['hits']['hits']))



9000


In [29]:
%%time

df = Select.from_dict(result_dict).to_pandas()
display(df)

,_index,_type,_id,_score,obj_id
0,articles,_doc,621612,1.0,621612
1,articles,_doc,414430,1.0,414430
2,articles,_doc,414899,1.0,414899
3,articles,_doc,740026,1.0,740026
4,articles,_doc,1747275,1.0,1747275
...,...,...,...,...,...
8995,articles,_doc,322835,1.0,322835
8996,articles,_doc,323364,1.0,323364
8997,articles,_doc,321121,1.0,321121
8998,articles,_doc,321603,1.0,321603


CPU times: user 44.6 ms, sys: 2.86 ms, total: 47.4 ms
Wall time: 45 ms
